In [1]:
%load_ext memory_profiler

In [2]:
import xarray as xr

# Performance

In [3]:
path = "/global/cfs/projectdirs/m4746/Datasets/ERA5/NA/2012/ERA5*.nc"
path = "/global/cfs/projectdirs/m4746/Datasets/ERA5/NA/2012/ERA5_2012-01.nc"

In [4]:
matching_files = [
    "/global/cfs/projectdirs/m4746/Datasets/ERA5/NA/2012/ERA5_2012-01.nc",
    "/global/cfs/projectdirs/m4746/Datasets/ERA5/NA/2012/ERA5_2012-02.nc",
    "/global/cfs/projectdirs/m4746/Datasets/ERA5/NA/2012/ERA5_2012-03.nc",
    "/global/cfs/projectdirs/m4746/Datasets/ERA5/NA/2012/ERA5_2012-04.nc",
    "/global/cfs/projectdirs/m4746/Datasets/ERA5/NA/2012/ERA5_2012-05.nc",
    "/global/cfs/projectdirs/m4746/Datasets/ERA5/NA/2012/ERA5_2012-06.nc",
    "/global/cfs/projectdirs/m4746/Datasets/ERA5/NA/2012/ERA5_2012-07.nc",
    "/global/cfs/projectdirs/m4746/Datasets/ERA5/NA/2012/ERA5_2012-08.nc",
    "/global/cfs/projectdirs/m4746/Datasets/ERA5/NA/2012/ERA5_2012-09.nc",
    "/global/cfs/projectdirs/m4746/Datasets/ERA5/NA/2012/ERA5_2012-10.nc",
    "/global/cfs/projectdirs/m4746/Datasets/ERA5/NA/2012/ERA5_2012-11.nc",
    "/global/cfs/projectdirs/m4746/Datasets/ERA5/NA/2012/ERA5_2012-12.nc",

]

## No Dask

### Concatenating 12 datasets

In [5]:
%%time
%%memit
       
ds_list = []
for file in matching_files:
    ds = xr.open_dataset(file, decode_times=True, chunks=None)
    ds_list.append(ds)

peak memory: 135.53 MiB, increment: 22.54 MiB
CPU times: user 147 ms, sys: 91.8 ms, total: 239 ms
Wall time: 5.31 s


In [6]:
print(f"Memory footprint: {ds_list[0].nbytes / 1e6:.2f} MB")

Memory footprint: 3748.18 MB


In [7]:
print(f"Memory footprint: {ds_list[-1].nbytes / 1e6:.2f} MB")

Memory footprint: 3748.18 MB


In [8]:
3748.18 * 12

44978.159999999996

In [9]:
%%time
%%memit

combine_kwargs = {
        "coords": "minimal",
        "compat": "override",
        "combine_attrs": "override",
}
ds = xr.combine_by_coords(ds_list, **combine_kwargs)

peak memory: 83799.49 MiB, increment: 83663.57 MiB
CPU times: user 23.2 s, sys: 37.4 s, total: 1min
Wall time: 11min 58s


In [10]:
ds

<xarray.Dataset> Size: 44GB
Dimensions:    (time: 8784, latitude: 241, longitude: 201)
Coordinates:
  * longitude  (longitude) float32 804B -50.0 -49.75 -49.5 ... -0.5 -0.25 0.0
  * latitude   (latitude) float32 964B 80.0 79.75 79.5 79.25 ... 20.5 20.25 20.0
  * time       (time) datetime64[ns] 70kB 2012-01-01 ... 2012-12-31T23:00:00
Data variables: (12/13)
    u10        (time, latitude, longitude) float64 3GB -1.67 -1.387 ... -2.35
    v10        (time, latitude, longitude) float64 3GB 5.594 5.553 ... -3.987
    d2m        (time, latitude, longitude) float64 3GB 224.1 224.0 ... 266.3
    t2m        (time, latitude, longitude) float64 3GB 228.2 228.1 ... 288.3
    msl        (time, latitude, longitude) float64 3GB 1.004e+05 ... 1.019e+05
    sst        (time, latitude, longitude) float64 3GB nan nan nan ... nan nan
    ...         ...
    ssr        (time, latitude, longitude) float64 3GB -2.328e-10 ... 2.328e-10
    str        (time, latitude, longitude) float64 3GB -1.948e+05 ... -3.47e+05
    sshf       (time, latitude, longitude) float64 3GB 9.895e+04 ... 1.118e+05
    ssrd       (time, latitude, longitude) float64 3GB 0.0 0.0 0.0 ... 0.0 0.0
    strd       (time, latitude, longitude) float64 3GB 3.384e+05 ... 1.002e+06
    tp         (time, latitude, longitude) float64 3GB 0.0 0.0 0.0 ... 0.0 0.0
Attributes:
    Conventions:  CF-1.6
    history:      2023-08-07 19:55:30 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [11]:
#ds.to_netcdf("/pscratch/sd/n/nloose/Datasets/ERA5/NA/2012/ERA5_2012.nc")

In [13]:
%%time
%%memit

combine_kwargs = {
        "coords": "minimal",
        "compat": "override",
        "combine_attrs": "override",
}
ds = xr.combine_nested(ds_list, **combine_kwargs, concat_dim="time")

peak memory: 126078.61 MiB, increment: 83716.71 MiB
CPU times: user 22 s, sys: 36.4 s, total: 58.4 s
Wall time: 5min 23s


In [14]:
print(f"Memory footprint: {ds.nbytes / 1e6:.2f} MB")

Memory footprint: 44252.67 MB


In [16]:
ds = ds.isel(time=slice(None, 744))

In [17]:
ds

<xarray.Dataset> Size: 4GB
Dimensions:    (time: 744, latitude: 241, longitude: 201)
Coordinates:
  * longitude  (longitude) float32 804B -50.0 -49.75 -49.5 ... -0.5 -0.25 0.0
  * latitude   (latitude) float32 964B 80.0 79.75 79.5 79.25 ... 20.5 20.25 20.0
  * time       (time) datetime64[ns] 6kB 2012-01-01 ... 2012-01-31T23:00:00
Data variables: (12/13)
    u10        (time, latitude, longitude) float64 288MB -1.67 -1.387 ... -2.703
    v10        (time, latitude, longitude) float64 288MB 5.594 5.553 ... -5.126
    d2m        (time, latitude, longitude) float64 288MB 224.1 224.0 ... 266.6
    t2m        (time, latitude, longitude) float64 288MB 228.2 228.1 ... 288.4
    msl        (time, latitude, longitude) float64 288MB 1.004e+05 ... 1.018e+05
    sst        (time, latitude, longitude) float64 288MB nan nan nan ... nan nan
    ...         ...
    ssr        (time, latitude, longitude) float64 288MB -2.328e-10 ... -2.32...
    str        (time, latitude, longitude) float64 288MB -1.948e+05 ... -3.28...
    sshf       (time, latitude, longitude) float64 288MB 9.895e+04 ... 1.229e+05
    ssrd       (time, latitude, longitude) float64 288MB 0.0 0.0 0.0 ... 0.0 0.0
    strd       (time, latitude, longitude) float64 288MB 3.384e+05 ... 1.042e+06
    tp         (time, latitude, longitude) float64 288MB 0.0 0.0 0.0 ... 0.0 0.0
Attributes:
    Conventions:  CF-1.6
    history:      2023-08-07 19:55:30 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [18]:
print(f"Memory footprint: {ds.nbytes / 1e6:.2f} MB")

Memory footprint: 3748.18 MB


### Only one dataset

In [1]:
from roms_tools.setup.datasets import ERA5Dataset

In [8]:
from datetime import datetime

In [7]:
ERA5Dataset?

Init signature:
ERA5Dataset(
    *,
    filename: Union[str, pathlib._local.Path, List[Union[str, pathlib._local.Path]]],
    start_time: Optional[datetime.datetime] = None,
    end_time: Optional[datetime.datetime] = None,
    dim_names: Dict[str, str] = <factory>,
    var_names: Dict[str, str] = <factory>,
    climatology: Optional[bool] = False,
    use_dask: Optional[bool] = False,
    apply_post_processing: Optional[bool] = True,
) -> None
Docstring:     
Represents ERA5 data on original grid.

Parameters
----------
filename : str
    The path to the data files. Can contain wildcards.
start_time : Optional[datetime], optional
    The start time for selecting relevant data. If not provided, the data is not filtered by start time.
end_time : Optional[datetime], optional
    The end time for selecting relevant data. If not provided, only data at the start_time is selected if start_time is provided,
    or no filtering is applied if start_time is not provided.
var_names: Dict[str, str

In [9]:
%%time
%%memit
data = ERA5Dataset(
    filename=matching_files[0],
    start_time=datetime(2012, 1, 1),
    end_time=datetime(2012, 1, 31)
)

peak memory: 2382.18 MiB, increment: 2183.25 MiB
CPU times: user 1.69 s, sys: 1.54 s, total: 3.23 s
Wall time: 13.3 s


In [10]:
data.ds

<xarray.Dataset> Size: 2GB
Dimensions:    (longitude: 201, latitude: 241, time: 721)
Coordinates:
  * longitude  (longitude) float32 804B -50.0 -49.75 -49.5 ... -0.5 -0.25 0.0
  * latitude   (latitude) float32 964B 20.0 20.25 20.5 20.75 ... 79.5 79.75 80.0
  * time       (time) datetime64[ns] 6kB 2012-01-01 ... 2012-01-31
Data variables:
    u10        (time, latitude, longitude) float64 279MB ...
    v10        (time, latitude, longitude) float64 279MB ...
    t2m        (time, latitude, longitude) float64 279MB 24.66 24.64 ... -4.621
    ssr        (time, latitude, longitude) float64 279MB -6.468e-14 ... -6.46...
    strd       (time, latitude, longitude) float64 279MB 368.2 368.6 ... 264.6
    tp         (time, latitude, longitude) float64 279MB 0.004609 ... 0.009218
    qair       (time, latitude, longitude) float64 279MB 0.01462 ... 0.002312
    mask       (latitude, longitude) int64 388kB 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1
Attributes:
    Conventions:  CF-1.6
    history:      2023-08-07 19:55:30 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

## Dask

In [22]:
    combine_kwargs = {
        "coords": "minimal",
        "compat": "override",
        "combine_attrs": "override",
    }

In [23]:
%%time
%%memit

ds = xr.open_mfdataset(
    matching_files,
    decode_times=True,
    chunks={"time": 1, "latitude": -1, "longitude": -1},
    combine="nested",
    concat_dim="time",
    **combine_kwargs,
)

peak memory: 84608.60 MiB, increment: 44.00 MiB
CPU times: user 273 ms, sys: 132 ms, total: 405 ms
Wall time: 3.4 s


In [24]:
%%time
%%memit

ds.load()

peak memory: 169590.52 MiB, increment: 84981.79 MiB
CPU times: user 1min 39s, sys: 55.8 s, total: 2min 35s
Wall time: 6min 55s


In [25]:
%%time
%%memit

ds = xr.open_mfdataset(
    matching_files,
    decode_times=True,
    chunks={"time": 1, "latitude": -1, "longitude": -1},
    combine="by_coords",
    **combine_kwargs,
)

peak memory: 159984.68 MiB, increment: 0.00 MiB
CPU times: user 302 ms, sys: 321 ms, total: 622 ms
Wall time: 3.66 s


In [26]:
%%time
%%memit

ds.load()

peak memory: 169573.09 MiB, increment: 51793.91 MiB
CPU times: user 1min 44s, sys: 1min 12s, total: 2min 56s
Wall time: 7min 32s


In [27]:
%%time
%%memit

ds = xr.open_mfdataset(
    matching_files,
    decode_times=True,
    chunks={"time": 1, "latitude": -1, "longitude": -1},
    combine="nested",
    concat_dim="time",
    **combine_kwargs,
)

peak memory: 160016.01 MiB, increment: 0.00 MiB
CPU times: user 293 ms, sys: 307 ms, total: 600 ms
Wall time: 2.74 s


In [28]:
%%time
%%memit

ds = ds.isel(time=slice(None, 744))

peak memory: 117808.53 MiB, increment: 0.00 MiB
CPU times: user 94.6 ms, sys: 208 ms, total: 302 ms
Wall time: 904 ms


In [29]:
%%time
%%memit

ds.load()

peak memory: 121415.79 MiB, increment: 3609.21 MiB
CPU times: user 8.86 s, sys: 6.73 s, total: 15.6 s
Wall time: 26.2 s


In [30]:
%%time
%%memit

ds = xr.open_mfdataset(
    matching_files,
    decode_times=True,
    chunks={"time": 1, "latitude": -1, "longitude": -1},
    combine="by_coords",
    **combine_kwargs,
)

peak memory: 121404.96 MiB, increment: 0.00 MiB
CPU times: user 315 ms, sys: 275 ms, total: 590 ms
Wall time: 2.47 s


In [31]:
%%time
%%memit

ds = ds.isel(time=slice(None, 744))

peak memory: 117829.44 MiB, increment: 0.00 MiB
CPU times: user 76.6 ms, sys: 233 ms, total: 309 ms
Wall time: 888 ms


In [32]:
%%time
%%memit

ds.load()

peak memory: 121403.98 MiB, increment: 3574.54 MiB
CPU times: user 8.62 s, sys: 7.1 s, total: 15.7 s
Wall time: 27.3 s
